# Fully Connected Autoencoder Example
modified from https://blog.keras.io/building-autoencoders-in-keras.html

## Prepare Data Set

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
(x_train, _),(x_test, _) = mnist.load_data()

## Check Data 

In [2]:
import matplotlib.pyplot as plt
print('x_train shape: {}  x_test shape: {}'.format(np.shape(x_train),np.shape(x_test)))
for i in range(16):
    plt.subplot(4, 4, 1 + i, xticks=[], yticks=[])
    img_id = np.random.randint(np.shape(x_train)[0])
    im = x_train[img_id,::]
    plt.imshow(im)
    plt.gray()

x_train shape: (60000, 28, 28)  x_test shape: (10000, 28, 28)


## Preprocessing Data

In [3]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


## Build Model 

In [4]:
from tensorflow.keras.layers import Dense
autoencoder = tf.keras.Sequential()
#Encoder
autoencoder.add(Dense(128,activation='relu',input_shape=(784,)))
autoencoder.add(Dense(64,activation='relu'))

#Compressed representation
autoencoder.add(Dense(32,activation='relu'))

#Decoder
autoencoder.add(Dense(64,activation='relu'))
autoencoder.add(Dense(128,activation='relu'))
autoencoder.add(Dense(784,activation='sigmoid'))

autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_4 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_5 (Dense)              (None, 784)               101136    
Total params: 222,384
Trainable params: 222,384
Non-trainable params: 0
_________________________________________________________________


## Train Model

In [ ]:
autoencoder.compile(optimizer='adadelta', loss='mse')

autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1142 - val_loss: 0.0692
Epoch 2/100
60000/60000 [==============================] - 2s 35us/step - loss: 0.0681 - val_loss: 0.0676
Epoch 3/100
60000/60000 [==============================] - 2s 35us/step - loss: 0.0667 - val_loss: 0.0661
Epoch 4/100
60000/60000 [==============================] - 2s 35us/step - loss: 0.0647 - val_loss: 0.0637
Epoch 5/100
60000/60000 [==============================] - 2s 35us/step - loss: 0.0622 - val_loss: 0.0616
Epoch 6/100
60000/60000 [==============================] - 2s 35us/step - loss: 0.0608 - val_loss: 0.0604
Epoch 7/100
60000/60000 [==============================] - 3s 43us/step - loss: 0.0597 - val_loss: 0.0592
Epoch 8/100
60000/60000 [==============================] - 2s 42us/step - loss: 0.0580 - val_loss: 0.0566
Epoch 9/100
60000/60000 [==============================] - 2s 35us/step - loss: 0.0547 - val_loss: 0.0

60000/60000 [==============================] - 3s 46us/step - loss: 0.0198 - val_loss: 0.0195
Epoch 78/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.0197 - val_loss: 0.0190
Epoch 79/100
60000/60000 [==============================] - 2s 36us/step - loss: 0.0196 - val_loss: 0.0194
Epoch 80/100
60000/60000 [==============================] - 2s 36us/step - loss: 0.0195 - val_loss: 0.0191
Epoch 81/100
60000/60000 [==============================] - 2s 35us/step - loss: 0.0194 - val_loss: 0.0188
Epoch 82/100
60000/60000 [==============================] - 2s 35us/step - loss: 0.0192 - val_loss: 0.0191
Epoch 83/100
60000/60000 [==============================] - 2s 35us/step - loss: 0.0192 - val_loss: 0.0185
Epoch 84/100
60000/60000 [==============================] - 2s 35us/step - loss: 0.0191 - val_loss: 0.0188
Epoch 85/100
60000/60000 [==============================] - 2s 38us/step - loss: 0.0190 - val_loss: 0.0187
Epoch 86/100
60000/60000 [========================

## Evaluate Model

In [ ]:
x_recon = autoencoder.predict(x_test)
for i in range(16):
    plt.subplot(4, 8, 1 + i*2, xticks=[], yticks=[])
    img_id = np.random.randint(np.shape(x_test)[0])
    im = x_test[img_id,::].reshape([28,28])
    plt.imshow(im)
    plt.gray()
    plt.title('Ori')
    plt.subplot(4, 8, 2 + i*2, xticks=[], yticks=[])
    
    im = x_recon[img_id,::].reshape([28,28])
    plt.imshow(im)
    plt.gray()
    plt.title('AE')

# Denosing with Autoencoder 

## Add noise to data

In [ ]:
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

## Check Data 

In [ ]:
for i in range(16):
    plt.subplot(4, 4, 1 + i, xticks=[], yticks=[])
    img_id = np.random.randint(np.shape(x_train)[0])
    im = x_train_noisy[img_id,::].reshape([28,28])
    plt.imshow(im)
    plt.gray()

## Create a New Model for Denosing

In [ ]:
tf.reset_default_graph()
tf.keras.backend.clear_session()
autoencoder = tf.keras.Sequential()
#Encoder
autoencoder.add(Dense(128,activation='relu',input_shape=(784,)))
autoencoder.add(Dense(64,activation='relu'))

#Compressed representation
autoencoder.add(Dense(32,activation='relu'))

#Decoder
autoencoder.add(Dense(64,activation='relu'))
autoencoder.add(Dense(128,activation='relu'))
autoencoder.add(Dense(784,activation='sigmoid'))

#Train
autoencoder.compile(optimizer='adadelta', loss='mse')

autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

## Evaluate Model

In [ ]:
x_denoise = autoencoder.predict(x_test_noisy)
for i in range(16):
    plt.subplot(4, 8, 1 + i*2, xticks=[], yticks=[])
    img_id = np.random.randint(np.shape(x_test_noisy)[0])
    im = x_test_noisy[img_id,::].reshape([28,28])
    plt.imshow(im)
    plt.gray()
    plt.title('Ori')
    plt.subplot(4, 8, 2 + i*2, xticks=[], yticks=[])
    
    im = x_denoise[img_id,::].reshape([28,28])
    plt.imshow(im)
    plt.gray()
    plt.title('AE')